In [37]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

1. Construir un clase  que permita definir una red neuronal con la topología
deseada y la función de activación para cada capa, para ello deberá construir una funcion Topology con el número de capas de la red neuronal :

Topology = [n_x, n_h1, n_h2, n_h3, ...,n_y]

También definir una lista con las funciones de activaciones para cada capa.

activation=[None, relu, relu, relu, ...,sigmoid]


a. Cada unas de las capas deberá tener los parámetros de inicialización de manera aleatoria:

La matriz de parametros para cada capa debera tener:


b. Construya un metodo llamado output cuya salida serán los valores de Z y A

In [38]:
#Función para las funciones de activacion:
def activation_functions(name):

    if name == "Linear":
        return lambda x: x, lambda x: np.ones_like(x)

    elif name == "Step":
        return lambda x: np.where(x >= 0, 1, 0), lambda x: np.where(x >= 0, 1, 0)

    elif name == "ReLU":
        return lambda x: np.maximum(0, x), lambda x: (x > 0).astype(float)

    elif name == "Leaky ReLU":
        alpha = 0.01
        return lambda x: np.where(x > 0, x, alpha * x), lambda x: np.where(x > 0, 1, alpha)

    elif name == "Sigmoid":
        return lambda x: 1 / (1 + np.exp(-x)), lambda x: (np.exp(-x)) / ((np.exp(-x) + 1) ** 2)

    elif name == "Tanh":
        return lambda x: np.tanh(x), lambda x: 1 - np.tanh**2

    elif name == "Softmax":
        return lambda x: np.exp(x) / np.sum(np.exp(x), axis=0), lambda x: np.exp(x) / np.sum(np.exp(x), axis=0)

    else:
        raise ValueError(f"Unknown activation function name: {name}")

In [39]:
# Clase para crear una capa
class layer_nn():

  def __init__(self,activation_function, neurons_number, before_neurons_number):
    self.activation_function = activation_functions(activation_function)
    self.neurons_number = neurons_number
    self.before_neurons_number = np.zeros((before_neurons_number,1))
    self.Theta = 2*np.random.randn(neurons_number,before_neurons_number)
    self.B = 2*np.random.randn(neurons_number,1)
    self.A = np.zeros((neurons_number,1))
    #self.Z = np.zeros((neurons_number,1))

  def print_parameters(self):
    print(f"Theta: \n{self.Theta}")
    print(f"B: \n{self.B}")

  def output(self, X):
    self.X = X
    self.Z = np.dot(self.Theta,X)+self.B
    self.A = self.activation_function[0](self.Z)
    return self.A, self.Z, self.Theta, self.B, X

  def update_parameters(self, new_Theta, new_B):
    self.Theta = new_Theta
    self.B = new_B

  def update_A(self, new_A):
    self.A = new_A

  def get_parameters(self):
    return self.A, self.Z, self.Theta, self.B, X

In [40]:
# Funcion para crear la red neuronal:
def create_NN(topology,function_names):

  if len(topology) != len(function_names):
    raise ValueError(f"Both lists must have the same length but: len(topology) = {len(topology)} and len(function_names) = {len(function_names)}")

  if any(x == 0 for x in topology):
    raise ValueError("Each layer must have at least one neuron")

  if any(x not in ["Linear","Step","ReLU","Leaky ReLU","Sigmoid","Tanh","Softmax"] for x in function_names):
    raise ValueError("Unknown activation function name")

  layers = []
  layers.append(layer_nn(function_names[0],topology[0],1))


  for i in range(1,len(topology)):
    layers.append(layer_nn(function_names[i],topology[i],topology[i-1]))
    #X = layers[i].output()[0]

  return layers

2. Construir un generalizacion de la red, en el que entrada el valor inicial
y la red neuronal completa arroje la salida y la actualizacion de la red con los parametros deseados:

In [835]:
def forward_pass(NN,A):

  for layer in NN:
    if NN.index(layer) != 0:
      A = layer.output(A)[0]

  return A

In [42]:
topology = [2,4,5,4,3]
function_names = ["Linear","ReLU","Leaky ReLU","ReLU","Sigmoid"]

X = np.random.randn(2,1)*3

NN_ = create_NN(topology,function_names)
output = forward_pass(NN_,X)
#np.shape(X)

#output = forward_pass(NN,X)

print(f"Output: \n{output}")
print("\n")

for layer in NN_:
  print(f"Layer {NN_.index(layer)+1}")
  layer.print_parameters()
  print("\n")

Output: 
[[1.12673213e-17]
 [1.00000000e+00]
 [6.52886468e-27]]


Layer 1
Theta: 
[[ 1.65816394]
 [-0.51929711]]
B: 
[[1.69900868]
 [0.56527628]]


Layer 2
Theta: 
[[-3.02822218  2.33422364]
 [-3.8302425  -0.04737864]
 [ 2.43322071 -0.58782382]
 [-2.48230663 -1.03735043]]
B: 
[[-0.29919418]
 [ 0.4152287 ]
 [ 0.47187441]
 [-0.9270015 ]]


Layer 3
Theta: 
[[-4.39761625 -2.71944956 -3.29269981 -2.30338217]
 [ 4.74466808  2.10389434 -0.24681086 -1.2945599 ]
 [ 0.24882804 -0.50772973 -1.74650089 -1.57048046]
 [-0.43131952  0.08616077  1.3646298  -1.40754135]
 [-1.80592219 -1.13968162 -1.59853704  0.41552202]]
B: 
[[-2.90544557]
 [ 2.95243035]
 [-2.05611498]
 [-0.01774236]
 [-3.37894871]]


Layer 4
Theta: 
[[-1.21516608  1.21782116  4.82132275 -1.89262424 -3.41359445]
 [ 3.52794445  2.50719191 -3.51709493  0.46328059  1.25064406]
 [ 6.12287261  0.2125848  -0.84563694 -2.67802716 -1.38076278]
 [ 2.82648188 -2.56973872  0.14146583  2.55770849  5.13941107]]
B: 
[[ 0.26724718]
 [-2.70305175]
 [ 

3. Encontrar la funcion de coste.


$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

In [43]:
def loss_function(y_data,y_predicted,m):
  y_data = np.array(y_data)
  y_predicted = np.clip(y_predicted, 1e-15, 1 - 1e-15)
  return -(1/m)*np.sum(y_data*np.log(y_predicted)+(1-y_data)*np.log(1-y_predicted))

Hallaremos la función coste para el dataset propuesto:

In [45]:
data_train= f"train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test= f"test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"],train_dataset["train_set_x"],train_dataset["train_set_y"]

test_classes, xtest,test_label =\
test_dataset["list_classes"],test_dataset["test_set_x"],test_dataset["test_set_y"]

xtrain_= np.reshape(xtrain,(209, 64*64*3))/255
xtest_ = np.reshape(xtest,(50, 64*64*3))/255

In [46]:
n = np.shape(xtrain_)[1]
topology = [n,4,5,4,1]
function_names = ["Linear","ReLU","Leaky ReLU","ReLU","Sigmoid"]
NN = create_NN(topology,function_names)
output = forward_pass(NN,xtrain_.T)
error = loss_function(train_label,output,np.shape(xtrain_)[0])
print("Loss: ",error)

Loss:  6.079183570643172


<ipython-input-38-4ff874b71dab>:18: RuntimeWarning: overflow encountered in exp
  return lambda x: 1 / (1 + np.exp(-x)), lambda x: (np.exp(-x)) / ((np.exp(-x) + 1) ** 2)


Como no se ha entrenado y los pesos se asignan aleatoriamente aproximadamente la mitad debe acertar.

4. Construir un codigo que permita realizar el BackwardPropagation:

En el código siguiente se encuentran todas las funciones definidas previamente incluyendo la función de retropropagación y la función para entrenar la red:

# Funciones:

In [48]:
#Función para las funciones de activacion:
def activation_functions(name):

    if name == "Linear":
        return lambda x: x, lambda x: np.ones_like(x)

    elif name == "Step":
        return lambda x: np.where(x >= 0, 1, 0), lambda x: np.where(x >= 0, 1, 0)

    elif name == "ReLU":
        return lambda x: np.maximum(0, x), lambda x: (x > 0).astype(float)

    elif name == "Leaky ReLU":
        alpha = 0.01
        return lambda x: np.where(x > 0, x, alpha * x), lambda x: np.where(x > 0, 1, alpha)

    elif name == "Sigmoid":
        #return lambda x: 1 / (1 + np.exp(-x)), lambda x: (np.exp(-x)) / ((np.exp(-x) + 1) ** 2)
        return lambda x: 1 / (1 + np.exp(-x)), lambda x: (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x))))
    elif name == "Tanh":
        return lambda x: np.tanh(x), lambda x: 1 - np.tanh(x)**2

    elif name == "Softmax":
        return lambda x: np.exp(x) / np.sum(np.exp(x), axis=0), lambda x: np.exp(x) / np.sum(np.exp(x), axis=0)

    else:
        raise ValueError(f"Unknown activation function name: {name}")




# Clase para crear una capa
class layer_nn():

  def __init__(self,activation_function, neurons_number, before_neurons_number):
    self.activation_function = activation_functions(activation_function)
    self.neurons_number = neurons_number
    self.before_neurons_number = np.zeros((before_neurons_number,1))
    self.Theta = np.sqrt(2. / before_neurons_number) * np.random.randn(neurons_number,before_neurons_number)
    self.B = np.zeros((neurons_number,1))
    self.A = np.zeros((neurons_number,1))
    #self.Z = np.zeros((neurons_number,1))

  def print_parameters(self):
    print(f"Theta: \n{self.Theta}")
    print(f"B: \n{self.B}")

  def output(self, X):
    self.X = X
    self.Z = np.dot(self.Theta,X)+self.B
    self.A = self.activation_function[0](self.Z)
    return self.A, self.Z, self.Theta, self.B, X

  def update_parameters(self, new_Theta, new_B):
    self.Theta = new_Theta
    self.B = new_B

  def update_A(self, new_A):
    self.A = new_A

  def get_parameters(self):
    return self.A, self.Z, self.Theta, self.B, self.X


# Funcion para crear la red neuronal:
def create_NN(topology,function_names):

  if len(topology) != len(function_names):
    raise ValueError(f"Both lists must have the same length but: len(topology) = {len(topology)} and len(function_names) = {len(function_names)}")

  if any(x == 0 for x in topology):
    raise ValueError("Each layer must have at least one neuron")

  if any(x not in ["Linear","Step","ReLU","Leaky ReLU","Sigmoid","Tanh","Softmax"] for x in function_names):
    raise ValueError("Unknown activation function name")

  layers = []
  layers.append(layer_nn(function_names[0],topology[0],1))


  for i in range(1,len(topology)):
    layers.append(layer_nn(function_names[i],topology[i],topology[i-1]))
    #X = layers[i].output()[0]

  return layers


# Propagacion hacia adelante:
def forward_pass(NN,A):
  for layer in NN:
    if NN.index(layer) != 0:
      A = layer.output(A)[0]
  return A

# Funcion de perdida de entropia cruzada:
def loss_function(y_data,y_predicted,m):
  y_data = np.array(y_data)
  y_predicted = np.clip(y_predicted, 1e-15, 1 - 1e-15)
  return -(1/m)*np.sum(y_data*np.log(y_predicted)+(1-y_data)*np.log(1-y_predicted))


def loss_function(y_data, y_predicted, m):
    y_data = np.array(y_data)
    y_predicted = np.clip(y_predicted, 1e-15, 1 - 1e-15)
    return -np.mean(y_data*np.log(y_predicted) + (1-y_data)*np.log(1-y_predicted))

# Backward propagation:
def backward_propagation(df,Theta, B, A, Z, dA_l, alpha, m):

  dZ = (dA_l*df(Z))
  #print(f"dZ: \n{dZ}")
  #print(f"dA_l: \n{dA_l}")
  #print(f"df: \n{df(Z)}")
  dTheta = 1/m * np.dot(A,dZ.T)
  dB = 1/m * np.sum(dZ, axis=1, keepdims=True)
  dA_1 = np.dot(Theta.T,dZ)

  Theta = Theta - alpha*dTheta.T
  B = B - alpha*dB

  return Theta, B, dA_1


# Entrenamiento:
def train(NN,X,Y,alpha,z):

  Y = np.array(Y)

  for i in range(z):
    output = forward_pass(NN,X)
    output = np.where(output < 0.5, 0, 1)
    #print(output[0])
    error = loss_function(Y,output[0],np.shape(X)[1])
    print("Loss: ",error)

    for j in range(len(NN)-1,0,-1):

      A = NN[j].get_parameters()[0]
      Z = NN[j].get_parameters()[1]
      Theta = NN[j].get_parameters()[2]
      B = NN[j].get_parameters()[3]
      df = NN[j].activation_function[1]

      if j == len(NN)-1:
        dA_l = ((Y)/(A+1e-10)) - ((1-Y)/(1-A+1e-10))
        dA_l = - (np.divide(Y, A) - np.divide(1 - Y, 1 - A))

      try:
        A = NN[j-1].get_parameters()[0]
      except:
        A = X
      NN[j].Theta, NN[j].B, dA_l = backward_propagation(df,Theta, B, A, Z, dA_l, alpha, np.shape(Y)[0])
  return NN

In [49]:
data_train= f"train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test= f"test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"],train_dataset["train_set_x"],train_dataset["train_set_y"]

test_classes, xtest,test_label =\
test_dataset["list_classes"],test_dataset["test_set_x"],test_dataset["test_set_y"]

xtrain_= np.reshape(xtrain,(209, 64*64*3))/255
xtrain_ = (np.reshape(xtrain, (209, 64*64*3)) - 127.5) / 127.5
xtest_ = np.reshape(xtest,(50, 64*64*3))/255
xtest_ = (np.reshape(xtest, (50, 64*64*3)) - 127.5) / 127.5

n = np.shape(xtrain_)[1]
topology = [n,4,5,4,1]
function_names = ["Linear","Tanh","Tanh","Tanh","Sigmoid"]
NN = create_NN(topology,function_names)
output = forward_pass(NN,xtrain_.T)
error = loss_function(train_label,output,np.shape(xtrain_)[0])
print("Loss: ",error)
train(NN,xtrain_.T,train_label,0.01,1000)

Loss:  0.7818371521150569
Loss:  16.52599431441986
Loss:  14.046981731544898
Loss:  13.551213647395128
Loss:  11.568114530020875
Loss:  12.39438956917046
Loss:  11.23762287832116
Loss:  11.898610007545615
Loss:  10.411324884221427
Loss:  11.733352704220684
Loss:  9.254512283471835
Loss:  10.741793580970993
Loss:  9.089251154321877
Loss:  10.080756716021218
Loss:  8.758732721846991
Loss:  8.923944115271624
Loss:  7.43664368864734
Loss:  7.932400295322034
Loss:  7.106125256172453
Loss:  7.601881862847146
Loss:  6.94087560449757
Loss:  7.601885688672171
Loss:  6.775618301172639
Loss:  8.262914901971898
Loss:  6.775622126997664
Loss:  8.262911076146873
Loss:  6.940887081972646
Loss:  8.262903424496823
Loss:  6.610372475322782
Loss:  7.436613082047141
Loss:  5.784066829573001
Loss:  5.949297352122758
Loss:  5.123022312973175
Loss:  5.949301177947783
Loss:  4.957765009648244
Loss:  5.618778919647871
Loss:  4.461981622198374
Loss:  5.1229993580230255
Loss:  4.627242751348331
Loss:  4.95773822

A continuación se restarán los datos etiquetados con los predichos, si el valor predicho es correcto debe salir un cero.

In [50]:
output = forward_pass(NN,xtest_.T)
output = np.where(output < 0.5, 0, 1)
output-test_label

array([[ 0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1, -1,  0,  1,  0,  0,
         0,  0, -1, -1,  0,  0,  0,  0, -1,  0,  0,  0, -1,  1, -1,  0,
         0, -1,  1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  1,  1, -1,  0,
         0,  0]])

In [51]:
output = forward_pass(NN,xtrain_.T)
output = np.where(output < 0.5, 0, 1)
output-train_label

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Aquí vemos que el entrenamiento hizo que la red se sobreajustara. Funciona muy bien con los datos de entrenamiento pero con los datos de testeo funciona casi igual que por azar. Intentemos disminuir los ciclos y aumentar la taza de aprendizaje.

In [53]:
n = np.shape(xtrain_)[1]
topology = [n,4,5,4,1]
function_names = ["Linear","Tanh","Tanh","Tanh","Sigmoid"]
NN_2 = create_NN(topology,function_names)
output = forward_pass(NN_2,xtrain_.T)
error = loss_function(train_label,output,np.shape(xtrain_)[0])
print("Loss: ",error)
train(NN_2,xtrain_.T,train_label,0.1,200)

Loss:  0.7086825508232768
Loss:  17.682833695944634
Loss:  12.229097833420305
Loss:  11.568053316820478
Loss:  11.402792187670522
Loss:  11.237534884345589
Loss:  11.072277581020659
Loss:  11.072277581020659
Loss:  10.741762974370797
Loss:  10.741762974370797
Loss:  10.576505671045863
Loss:  10.576505671045863
Loss:  10.576505671045863
Loss:  10.576505671045863
Loss:  10.576505671045863
Loss:  10.245979586920926
Loss:  9.750207676946133
Loss:  9.915468806096088
Loss:  9.915464980271064
Loss:  9.915464980271064
Loss:  9.419693070296269
Loss:  9.5849503736212
Loss:  9.419693070296269
Loss:  9.089186115296457
Loss:  8.758663856996543
Loss:  9.089178463646407
Loss:  8.42814925034668
Loss:  8.923921160321475
Loss:  7.601858907896998
Loss:  8.758660031171518
Loss:  14.708141022895468
Loss:  9.089178463646407
Loss:  8.758663856996543
Loss:  8.428145424521656
Loss:  8.097630817871792
Loss:  5.949285874647683
Loss:  7.106086997922203
Loss:  13.881885112871009
Loss:  9.089178463646407
Loss:  8.5

In [54]:
output = forward_pass(NN_2,xtest_.T)
output = np.where(output < 0.5, 0, 1)
output-test_label

array([[ 0,  0,  0,  0, -1,  0, -1,  0,  0, -1, -1, -1,  0,  1,  0, -1,
         0,  0, -1, -1,  0,  0,  0,  0,  0, -1,  0,  0, -1,  1, -1,  0,
         0, -1,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0, -1,  0,
         0,  0]])

In [55]:
output = forward_pass(NN_2,xtrain_.T)
output = np.where(output < 0.5, 0, 1)
output-train_label

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]])

Hay una leve mejora con los datos de testeo.

In [69]:
n = np.shape(xtrain_)[1]
topology = [n,4,5,4,1]
function_names = ["Linear","Tanh","Tanh","Tanh","Sigmoid"]
NN_3 = create_NN(topology,function_names)
output = forward_pass(NN_3,xtrain_.T)
error = loss_function(train_label,output,np.shape(xtrain_)[0])
print("Loss: ",error)
train(NN_3,xtrain_.T,train_label,0.1,50)

Loss:  0.7528301849582856
Loss:  21.483847318043676
Loss:  13.220660782495017
Loss:  12.559627743370266
Loss:  12.063844355920398
Loss:  11.568060968470528
Loss:  11.402799839320572
Loss:  11.402796013495546
Loss:  11.072281406845685
Loss:  10.907020277695725
Loss:  10.741762974370797
Loss:  10.411244541895908
Loss:  10.245987238570976
Loss:  10.080729935246044
Loss:  9.750215328596182
Loss:  9.419693070296269
Loss:  9.254424289496264
Loss:  9.584942721971151
Loss:  8.593383598721463
Loss:  9.089166986171332
Loss:  9.254424289496264
Loss:  7.932354385421736
Loss:  8.097611688746667
Loss:  8.097615514571693
Loss:  7.601839778771874
Loss:  8.923913508671426
Loss:  8.593402727846588
Loss:  7.932358211246761
Loss:  7.932354385421736
Loss:  8.262880469546674
Loss:  7.601855082071973
Loss:  6.6102921329972615
Loss:  6.940814391297173
Loss:  7.271328997947036
Loss:  7.1060755204471295
Loss:  7.767154469472178
Loss:  7.4365863012719675
Loss:  6.44503482967233
Loss:  7.271336649597085
Loss:  6.

In [70]:
output = forward_pass(NN_3,xtest_.T)
output = np.where(output < 0.5, 0, 1)
output-test_label

array([[ 0,  0,  0, -1, -1,  0, -1,  0,  0, -1,  0, -1,  0,  0,  0,  0,
         0,  0, -1, -1,  0,  0,  0, -1,  0,  0, -1,  0, -1,  1, -1, -1,
         0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,
        -1,  0]])

In [71]:
output = forward_pass(NN_3,xtrain_.T)
output = np.where(output < 0.5, 0, 1)
output-train_label

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  1,
         0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
         0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
         0]])